In [ ]:
# Copy dataset from Drive to local content directory
!cp -r /content/drive/MyDrive/dakshina_dataset_v1.0 /content/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pandas as pd
import numpy as np
import wandb
import os
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


## Build Vocabulary
Take the English and hindi words to make a vocabulary. T
The English words will be used to transliterate the Hindi words.

In [ ]:
# Choose your language
LANG = 'hi'  # Hindi
DATA_DIR = f'/content/dakshina_dataset_v1.0/{LANG}/lexicons/'

def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    input_texts, target_texts = [], []
    for line in lines:
        latin, devanagari = line.strip().split('\t')[:2]
        input_texts.append(list(latin.lower()))
        target_texts.append(['<sos>'] + list(devanagari) + ['<eos>'])
    return input_texts, target_texts

train_x, train_y = load_data(os.path.join(DATA_DIR, 'hi.translit.sampled.train.tsv'))
val_x, val_y = load_data(os.path.join(DATA_DIR, 'hi.translit.sampled.dev.tsv'))
test_x, test_y = load_data(os.path.join(DATA_DIR, 'hi.translit.sampled.test.tsv'))

# Create vocab
def build_vocab(seqs):
    vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    for seq in seqs:
        for ch in seq:
            if ch not in vocab:
                vocab[ch] = len(vocab)
    return vocab

src_vocab = build_vocab(train_x)
tgt_vocab = build_vocab(train_y)

inv_tgt_vocab = {v: k for k, v in tgt_vocab.items()}


In [ ]:
class TransliterationDataset(Dataset):
    def __init__(self, src_data, tgt_data, src_vocab, tgt_vocab):
        self.src_data = src_data
        self.tgt_data = tgt_data
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_seq = [self.src_vocab[ch] for ch in self.src_data[idx]]
        tgt_seq = [self.tgt_vocab[ch] for ch in self.tgt_data[idx]]
        return torch.tensor(src_seq), torch.tensor(tgt_seq)

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_lens = torch.tensor([len(s) for s in src_batch])
    tgt_lens = torch.tensor([len(t) for t in tgt_batch])
    src_batch = pad_sequence(src_batch, padding_value=0, batch_first=True)
    tgt_batch = pad_sequence(tgt_batch, padding_value=0, batch_first=True)
    return src_batch, tgt_batch, src_lens, tgt_lens

BATCH_SIZE = 64
#Prepare train loader
train_dataset = TransliterationDataset(train_x, train_y, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
#Prepare val loader
val_dataset = TransliterationDataset(val_x, val_y, src_vocab, tgt_vocab)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# Prepare test loader
test_dataset = TransliterationDataset(test_x, test_y, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Model Definition
A Seq2Seq model is constructed with encoder decoder logic.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder_embed = nn.Embedding(config['src_vocab_size'], config['embedding_size'])
        self.decoder_embed = nn.Embedding(config['tgt_vocab_size'], config['embedding_size'])

        cell = {'RNN': nn.RNN, 'GRU': nn.GRU, 'LSTM': nn.LSTM}[config['cell_type']]

        self.encoder = cell(config['embedding_size'], config['hidden_size'], config['num_encoder_layers'],
                            batch_first=True, dropout=config['dropout'] if config['num_encoder_layers'] > 1 else 0)

        self.decoder = cell(config['embedding_size'], config['hidden_size'], config['num_decoder_layers'],
                            batch_first=True, dropout=config['dropout'] if config['num_decoder_layers'] > 1 else 0)

        self.fc = nn.Linear(config['hidden_size'], config['tgt_vocab_size'])

        self.cell_type = config['cell_type']

    def forward(self, src, tgt, src_len, tgt_len):
        src_embed = self.encoder_embed(src)
        tgt_embed = self.decoder_embed(tgt[:, :-1])

        _, h = self.encoder(src_embed)

        if self.cell_type == 'LSTM':
            h0, c0 = h
            output, _ = self.decoder(tgt_embed, (h0, c0))
        else:
            output, _ = self.decoder(tgt_embed, h)

        logits = self.fc(output)
        return logits


## Sweeping

Function used to perform hyperparameter sweep

In [ ]:
def train_model(train_loader, val_loader, config=None):
    with wandb.init(config=config) as run:
        config = wandb.config

        # Skip invalid layer configs
        if config.num_encoder_layers != config.num_decoder_layers:
            print(f"Skipping run due to layer mismatch: enc={config.num_encoder_layers}, dec={config.num_decoder_layers}")
            return

        # Set run name from config
        run.name = (
            f"ed_{config.embedding_size}"
            f"_hs_{config.hidden_size}"
            f"_enc_{config.num_encoder_layers}"
            f"_dec_{config.num_decoder_layers}"
            f"_cell_{config.cell_type}"
            f"_drop_{config.dropout}"
        )
        run.save()

        # Create model
        model = Seq2Seq({
            'src_vocab_size': len(src_vocab),
            'tgt_vocab_size': len(tgt_vocab),
            'embedding_size': config.embedding_size,
            'hidden_size': config.hidden_size,
            'num_encoder_layers': config.num_encoder_layers,
            'num_decoder_layers': config.num_decoder_layers,
            'cell_type': config.cell_type,
            'dropout': config.dropout
        }).to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss(ignore_index=0)

        best_val_acc = 0
        for epoch in range(10):
            # === Training ===
            model.train()
            total_train_loss = 0
            correct_train = 0
            total_tokens = 0

            for src, tgt, src_len, tgt_len in train_loader:
                src, tgt = src.to(device), tgt.to(device)
                optimizer.zero_grad()

                output = model(src, tgt, src_len, tgt_len)
                loss = criterion(output.view(-1, output.shape[-1]), tgt[:, 1:].contiguous().view(-1))
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()
                pred = output.argmax(-1)
                mask = tgt[:, 1:] != 0
                correct_train += ((pred == tgt[:, 1:]) * mask).sum().item()
                total_tokens += mask.sum().item()

            train_loss = total_train_loss / len(train_loader)
            train_acc = correct_train / total_tokens

            # === Validation ===
            model.eval()
            total_val_loss = 0
            correct_val = 0
            val_tokens = 0

            with torch.no_grad():
                for src, tgt, src_len, tgt_len in val_loader:
                    src, tgt = src.to(device), tgt.to(device)
                    output = model(src, tgt, src_len, tgt_len)
                    loss = criterion(output.view(-1, output.shape[-1]), tgt[:, 1:].contiguous().view(-1))
                    total_val_loss += loss.item()

                    pred = output.argmax(-1)
                    mask = tgt[:, 1:] != 0
                    correct_val += ((pred == tgt[:, 1:]) * mask).sum().item()
                    val_tokens += mask.sum().item()

            val_loss = total_val_loss / len(val_loader)
            val_acc = correct_val / val_tokens

            # Log to wandb
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_accuracy': train_acc,
                'val_accuracy': val_acc
            })

        # Print final metrics
        print(f"\nFinal Run Metrics for {run.name}:")
        print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Val   Accuracy: {val_acc:.4f}\n")

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_loss', 'goal': 'minimize'},
    'parameters': {
        'embedding_size': {'values': [16, 32, 64, 256]},
        'hidden_size': {'values': [16, 32, 64, 256]},
        'num_encoder_layers': {'values': [1, 2, 3]},
        'num_decoder_layers': {'values': [1, 2, 3]},
        'cell_type': {'values': ['RNN', 'GRU', 'LSTM']},
        'dropout': {'values': [0.2, 0.3]},
        'beam_size': {'values': [1, 2, 3, 5, 10]}
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Assignment3_Attempt1")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ex4t3obv
Sweep URL: https://wandb.ai/vinyk-sd-indian-institute-of-technology-madras/Assignment3_Attempt1/sweeps/ex4t3obv


In [ ]:
wandb.agent(sweep_id, function=lambda: train_model(train_loader, val_loader), count=50)# Run 15 trials (you can increase if needed)

wandb: Agent Starting Run: qvetco5x with config:
wandb: 	beam_size: 3
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embedding_size: 32
wandb: 	hidden_size: 32
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2
wandb: Currently logged in as: vinyk-sd (vinyk-sd-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Skipping run due to layer mismatch: enc=2, dec=1


wandb: Agent Starting Run: qi5j61zd with config:
wandb: 	beam_size: 2
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	hidden_size: 64
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2


Skipping run due to layer mismatch: enc=2, dec=1


wandb: Agent Starting Run: n97uybr5 with config:
wandb: 	beam_size: 2
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	embedding_size: 32
wandb: 	hidden_size: 32
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 2


...

Skipping run due to layer mismatch: enc=1, dec=3


wandb: Agent Starting Run: 41p8iwks with config:
wandb: 	beam_size: 1
wandb: 	cell_type: RNN
wandb: 	dropout: 0.3
wandb: 	embedding_size: 256
wandb: 	hidden_size: 32
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2


Skipping run due to layer mismatch: enc=2, dec=1


wandb: Agent Starting Run: sezmfcya with config:
wandb: 	beam_size: 5
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	hidden_size: 32
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1


Skipping run due to layer mismatch: enc=1, dec=3


In [ ]:
wandb.agent(sweep_id, function=lambda: train_model(train_loader, val_loader), count=150)# Run 15 trials (you can increase if needed)

wandb: Agent Starting Run: nxay9hod with config:
wandb: 	beam_size: 3
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	hidden_size: 64
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 1


Skipping run due to layer mismatch: enc=1, dec=3


wandb: Agent Starting Run: 8y2sqfw4 with config:
wandb: 	beam_size: 10
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embedding_size: 256
wandb: 	hidden_size: 256
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 3



Final Run Metrics for ed_256_hs_256_enc_3_dec_3_cell_GRU_drop_0.3:
Train Loss: 0.2889 | Train Accuracy: 0.8971
Val   Loss: 0.3582 | Val   Accuracy: 0.8865



epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▆▇▇██████
train_loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▁▅▇▇▇▇████
val_loss,█▄▃▂▁▂▁▁▁▁
epoch,9
train_accuracy,0.89711
train_loss,0.28888
val_accuracy,0.88655
val_loss,0.35819


...

Skipping run due to layer mismatch: enc=2, dec=1


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: mx8kad0y with config:
wandb: 	beam_size: 10
wandb: 	cell_type: GRU
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	hidden_size: 32
wandb: 	num_decoder_layers: 1
wandb: 	num_encoder_layers: 2


Skipping run due to layer mismatch: enc=2, dec=1


wandb: Agent Starting Run: hcek71ns with config:
wandb: 	beam_size: 2
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	hidden_size: 16
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 2


Skipping run due to layer mismatch: enc=2, dec=3
